In [0]:
!pip install -U -q kaggle
!mkdir -p ~/.kaggle

In [0]:
from google.colab import files
files.upload()

In [3]:
!cp kaggle.json ~/.kaggle/
!kaggle datasets download -d lantian773030/pokemonclassification

100% 417M/417M [00:10<00:00, 49.8MB/s]
100% 417M/417M [00:10<00:00, 39.9MB/s]


In [4]:
import os
from zipfile import ZipFile
with ZipFile('pokemonclassification.zip', 'r') as zipObj:
   # Extract all the contents of zip file in the data directory
   zipObj.extractall('./data/')
print("done")

done


In [5]:
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt

from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.preprocessing.image import img_to_array
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D, AveragePooling2D, UpSampling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras import backend as K
import keras

from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

from imutils import paths
import numpy as np
import argparse
import random
import pickle
import cv2
import os

Using TensorFlow backend.


In [0]:
def cnn_():
    model = Sequential()
    chanDim = -1

    model.add(Conv2D(32, (3, 3), padding="same",
	    input_shape=image_dim, activation='relu'))
    model.add(BatchNormalization(axis=chanDim))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding="same", activation='relu'))
    model.add(BatchNormalization(axis=chanDim))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(128, (3, 3), padding="same", activation='relu'))
    model.add(BatchNormalization(axis=chanDim))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(1024))
    model.add(Activation("relu"))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))

		# softmax classifier
    model.add(Dense(len(lb.classes_)))
    model.add(Activation("softmax"))

    return model

In [0]:
e = 200
lr = 0.001
bs = 32
image_dim = (128,128,3)

In [8]:
data = []
labels = []

print("Loading images : ")
pokemon_paths = list(paths.list_images("data"))
pokemon_paths = sorted(pokemon_paths)
now_pokemon = ''
for pokemon_path in pokemon_paths:
    pokemon_name = pokemon_path.split(os.path.sep)[-2]

    if(pokemon_name != now_pokemon):
        now_pokemon = pokemon_name
        print(f"reading {now_pokemon}'s images")

    image = cv2.imread(pokemon_path)
    image = cv2.resize(image, (image_dim[1], image_dim[0]))
    image = img_to_array(image)

    data.append(image)
    labels.append(pokemon_name)
    

Loading images : 
reading Abra's images
reading Aerodactyl's images
reading Alakazam's images
reading Alolan Sandslash's images
reading Arbok's images
reading Arcanine's images
reading Articuno's images
reading Beedrill's images
reading Bellsprout's images
reading Blastoise's images
reading Bulbasaur's images
reading Butterfree's images
reading Caterpie's images
reading Chansey's images
reading Charizard's images
reading Charmander's images
reading Charmeleon's images
reading Clefable's images
reading Clefairy's images
reading Cloyster's images
reading Cubone's images
reading Dewgong's images
reading Diglett's images
reading Ditto's images
reading Dodrio's images
reading Doduo's images
reading Dragonair's images
reading Dragonite's images
reading Dratini's images
reading Drowzee's images
reading Dugtrio's images
reading Eevee's images
reading Ekans's images
reading Electabuzz's images
reading Electrode's images
reading Exeggcute's images
reading Exeggutor's images
reading Farfetchd's i

In [0]:
data = np.array(data) /255.0
labels = np.array(labels)

In [0]:
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.2, random_state=69)

In [0]:
aug = ImageDataGenerator(rotation_range=25, width_shift_range=0.1,
	height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
	horizontal_flip=True, fill_mode="nearest")

In [12]:
print("model create :")
model = cnn_()
model.summary()

model create :
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 128, 128, 32)      896       
_________________________________________________________________
batch_normalization_1 (Batch (None, 128, 128, 32)      128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 64)        18496     
_________________________________________________________________
batch_normalization_2 (Batch (None, 64, 64, 64)        256       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 32,

In [13]:
print("compile model :")
model.compile(loss=keras.losses.categorical_crossentropy, 
    optimizer=keras.optimizers.Adam(lr=lr), 
    metrics=['accuracy'])

compile model :


In [18]:
print("training step :")
print(trainX.shape)

training_history = model.fit(trainX, trainY, bs, e, validation_data=(testX,testY))

training step :
(5456, 128, 128, 3)
Train on 5456 samples, validate on 1364 samples
Epoch 1/200
5456/5456 [==============================] - 14s 3ms/step - loss: 0.0400 - accuracy: 0.9956 - val_loss: 2.2886 - val_accuracy: 0.5755
Epoch 2/200
5456/5456 [==============================] - 14s 3ms/step - loss: 0.0439 - accuracy: 0.9943 - val_loss: 1.9192 - val_accuracy: 0.5792
Epoch 3/200
5456/5456 [==============================] - 14s 3ms/step - loss: 0.0420 - accuracy: 0.9930 - val_loss: 2.0236 - val_accuracy: 0.6019
Epoch 4/200
5456/5456 [==============================] - 14s 3ms/step - loss: 0.0550 - accuracy: 0.9890 - val_loss: 2.1171 - val_accuracy: 0.5777
Epoch 5/200
5456/5456 [==============================] - 14s 3ms/step - loss: 0.0470 - accuracy: 0.9912 - val_loss: 2.5198 - val_accuracy: 0.5411
Epoch 6/200
5456/5456 [==============================] - 14s 3ms/step - loss: 0.0466 - accuracy: 0.9925 - val_loss: 3.9588 - val_accuracy: 0.4707
Epoch 7/200
5456/5456 [=================

In [0]:
model.save("pokemon_classifier_model.h5")